<html>
    <summary></summary>
         <div> <p></p> </div>
         <div style="font-size: 20px; width: 800px;"> 
              <h1>
               <left>Downloading and Annotating Genomic Sequences</left>
              </h1>
              <p><left>============================================================================</left> </p>
<pre>Course: BIOM/CBE 480A5, Spring 2025
Instructor: Brian Munsky
Contact Info: munsky@colostate.edu
</pre>
         </div>
    </p>

</html>

<details>
  <summary>Copyright info</summary>

```
Copyright 2024 Brian Munsky

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
```
<details>



# **Learning Objectives**
In this lesson, we are going to use some common sequence manipulation and search tools to learn about the DNA, RNA and proteins related to the Corona Virus. After completing this lesson, you will be able to:

* Describe the meaning and use of ```accession numbers``` to identify biological data sets.
* Describe and use Python tools to acquire genomic data from online ```repositories```
* Find and describe basic information that is commonly used to ```annotate``` genetic sequences
* Find and analyze ```open reading frames``` in DNA and RNA sequences
* ```Translate``` between DNA, cDNA, RNA and protein sequences
* Find and use different software packages to ```analyze``` biological sequence data
* Access online electronic information and primary literature to gain more information about specific biological sequences and data

# **1) Downloading genomic data from an online database**

Before we can analyze any given sequence data set, the first step is always going to be to obtain the data!

In this notebook, we are goin to focus in on a specific example, namely, the **corona virus**.

Our first step is to download the corona virus sequence (and a large amount of information about that sequence) from the ```genbank``` data base.  

I did a quick internet seach and found that an accession number from Severe acute respiratory syndrome coronavirus 2 isolate  Wuhan-Hu-1, complete genome: ```MN908947```.

We are going to use the modules ```Bio``` and ```biopython```, so let's import those now. 

**Note** - If any of the following packages do not import, you may need to install the missing modules using magic commands (see code below).

In [1]:
# Note, if you have not already installed the following packages, you can do so
#  by running the following terminal commands.  Note, you only need to run these
#  commands once.  If you have already installed these packages, you can skip
#  this step.
# !pip install biopython
# !pip install Bio

# Importing the necessary libraries
import Bio
from Bio import Entrez, SeqIO, pairwise2, AlignIO, Phylo
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils import ProtParam
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import webbrowser
import os
import threading

/Users/munsky/anaconda3/envs/BIOM480A5/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


The following code block will set up your user account in NCBU and then download the sequence data for the corona virus. 

In [2]:
#(PRELIMINARIES) Let's download the COVID19 virus sequence from NCBI
# Provide your email for accessing NCBI
Entrez.email = "brian.munsky@colostate.edu"  # Insert your email here

def get_genbank(accession_number):
    handle = Entrez.efetch(db="nucleotide", id=accession_number, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    return record

corona_virus = get_genbank("MN908947")

# **2) Exploring features of genomic data**

Now that we have the data, let's explore to see what information is available to us:

In [3]:
# Let's look at what kinds of information are available to use in the data set:
print(corona_virus)

ID: MN908947.3
Name: MN908947
Description: Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
Number of features: 23
/molecule_type=ss-RNA
/topology=linear
/data_file_division=VRL
/date=18-MAR-2020
/accessions=['MN908947']
/sequence_version=3
/keywords=['']
/source=Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)
/organism=Severe acute respiratory syndrome coronavirus 2
/taxonomy=['Viruses', 'Riboviria', 'Orthornavirae', 'Pisuviricota', 'Pisoniviricetes', 'Nidovirales', 'Cornidovirineae', 'Coronaviridae', 'Orthocoronavirinae', 'Betacoronavirus', 'Sarbecovirus', 'Severe acute respiratory syndrome-related coronavirus']
/references=[Reference(title='A new coronavirus associated with human respiratory disease in China', ...), Reference(title='Direct Submission', ...)]
/comment=On Jan 17, 2020 this sequence version replaced MN908947.2.
/structured_comment=defaultdict(<class 'dict'>, {'Assembly-Data': {'Assembly Method': 'Megahit v. V1.1.3', 'Sequ

### **2.1) Accession Number**

The first thing we can do is to print the accession number of the sequence. This is a unique identifier for the sequence in the database.

In [4]:
# Print the accession number
print(corona_virus.id)

MN908947.3


### **2.2) Sequence**

Next, we can print the sequence itself. This is the actual DNA sequence that we will be analyzing.

In [5]:
# Print the first 1000 nucleotides.
print(corona_virus.seq[0:1000])         # Output the sequence to screen.

ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTGTTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGATGCTCGAACTGCACCTCATGGTCATGTTATGGTTGAGCTGGTAGCAGAACTCGAAGGCATTCAGTACGGTCGTAGTGGTGAGACACTTGGTGTCCTTGTCCCTCATGTGGGCGAAATACCAGTGGCTTACCGCAAGGTTCTTCTTCGTAAGAACGGTAATAAAGGAGCTGGTGGCCATAGTTACGGCGCCGATCTAAAGTCATTTGACTTAGGCGACGAGCTTGGCACTGATCCTTATGAAGATTTTCAAGAAAACTGGAACACTAAACATAGCAGTGGTGTTACCCGTGAACTCATGCGTGAGCTTAACGGAGGGGCATACACTCGCTATGTCGATAACAACTTCTGTGGCCCTGATGGCTACCCTCTTGAGTGCATTAAAGACCTTCTAGCACGTGCTGGTAAAGCTTCATGCACTTTGTCCGAACAACTGGACTTTATTGACACTAAGAGGGGTGTATACTGCTGCCGTGAACATGAGCATGAAATTGCTTGGTACACGGAACGTTCT

In [6]:
# Find the length of the sequence
len(corona_virus.seq)  # Find the length of the sequence.

29903

## **2.3) Description**

The description of the sequence is a human-readable description of the sequence. This can include information about the organism the sequence comes from, the gene the sequence codes for, and other relevant information.

In [7]:
# Print description of the sequence
print(corona_virus.description)

Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome


## **2.4) Annotations**

Annotations are additional information about the sequence that can be used to help interpret the sequence. This can include information about the gene the sequence codes for, the location of the gene on the chromosome, and other relevant information.

In [8]:
# Explore what annotations are made about the sequence
corona_virus.annotations

{'molecule_type': 'ss-RNA',
 'topology': 'linear',
 'data_file_division': 'VRL',
 'date': '18-MAR-2020',
 'accessions': ['MN908947'],
 'sequence_version': 3,
 'keywords': [''],
 'source': 'Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)',
 'organism': 'Severe acute respiratory syndrome coronavirus 2',
 'taxonomy': ['Viruses',
  'Riboviria',
  'Orthornavirae',
  'Pisuviricota',
  'Pisoniviricetes',
  'Nidovirales',
  'Cornidovirineae',
  'Coronaviridae',
  'Orthocoronavirinae',
  'Betacoronavirus',
  'Sarbecovirus',
  'Severe acute respiratory syndrome-related coronavirus'],
 'references': [Reference(title='A new coronavirus associated with human respiratory disease in China', ...),
  Reference(title='Direct Submission', ...)],
 'comment': 'On Jan 17, 2020 this sequence version replaced MN908947.2.',
 'structured_comment': defaultdict(dict,
             {'Assembly-Data': {'Assembly Method': 'Megahit v. V1.1.3',
               'Sequencing Technology': 'Illumina'}})}

In [9]:
# What is the taxonomy of the sequence?
corona_virus.annotations['taxonomy']

['Viruses',
 'Riboviria',
 'Orthornavirae',
 'Pisuviricota',
 'Pisoniviricetes',
 'Nidovirales',
 'Cornidovirineae',
 'Coronaviridae',
 'Orthocoronavirinae',
 'Betacoronavirus',
 'Sarbecovirus',
 'Severe acute respiratory syndrome-related coronavirus']

## **2.5) Exploring advanced features in the genome**

Here is an example for how to download the genbank file to disk so that you can analyse it later.

In [10]:
def download_genbank(accession_number, output_file):
    try:
        # Fetch the GenBank record
        handle = Entrez.efetch(db="nucleotide", id=accession_number, rettype="gb", retmode="text")

        # Write the record to a file
        with open(output_file, "w") as f:
            f.write(handle.read())

        print(f"GenBank record downloaded and saved to {output_file}")

    except Exception as e:
        print(f"Error: {e}")

# Example: Download GenBank record for a given accession number
accession_number = "MN908947"  # Replace with your accession number
output_file = "coronaVirus.gb"

download_genbank(accession_number, output_file)

GenBank record downloaded and saved to coronaVirus.gb


Now that we have the full genbank file, let's explore some of the features of the genome. 

In [11]:
# Now that the file is saved, you can re-open it to get the information.
def extract_features(genbank_file):
    extractRecord = []
    for record in SeqIO.parse(genbank_file, 'genbank'):
        print(f"ID: {record.id}")
        for feature in record.features:
            print(f"Feature Type: {feature.type}")
            print(f"Location: {feature.location}")
            print(f"Qualifiers: {feature.qualifiers}")
        extractRecord.append(record)
    return extractRecord

extractedRecord = extract_features(output_file)

ID: MN908947.3
Feature Type: source
Location: [0:29903](+)
Qualifiers: {'organism': ['Severe acute respiratory syndrome coronavirus 2'], 'mol_type': ['genomic RNA'], 'isolate': ['Wuhan-Hu-1'], 'host': ['Homo sapiens'], 'db_xref': ['taxon:2697049'], 'geo_loc_name': ['China'], 'collection_date': ['Dec-2019']}
Feature Type: 5'UTR
Location: [0:265](+)
Qualifiers: {}
Feature Type: gene
Location: [265:21555](+)
Qualifiers: {'gene': ['orf1ab']}
Feature Type: CDS
Location: join{[265:13468](+), [13467:21555](+)}
Qualifiers: {'gene': ['orf1ab'], 'ribosomal_slippage': [''], 'note': ['pp1ab; translated by -1 ribosomal frameshift'], 'codon_start': ['1'], 'product': ['orf1ab polyprotein'], 'protein_id': ['QHD43415.1'], 'translation': ['MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNG

In [12]:
# Let's see what kinds of features there are in the data set
featureTypes = set([feature.type for feature in extractedRecord[0].features])
print(f'Types of features: {featureTypes}')

Types of features: {'source', 'CDS', "5'UTR", "3'UTR", 'gene'}


In [13]:
# Let's focus on coding sequences (CDS) and see what information is available
for i, feature in enumerate(extractedRecord[0].features):
    if feature.type == 'CDS':
        print(f"CDS Feature {i}: {feature.qualifiers.keys()}")

CDS Feature 3: dict_keys(['gene', 'ribosomal_slippage', 'note', 'codon_start', 'product', 'protein_id', 'translation'])
CDS Feature 5: dict_keys(['gene', 'note', 'codon_start', 'product', 'protein_id', 'translation'])
CDS Feature 7: dict_keys(['gene', 'codon_start', 'product', 'protein_id', 'translation'])
CDS Feature 9: dict_keys(['gene', 'note', 'codon_start', 'product', 'protein_id', 'translation'])
CDS Feature 11: dict_keys(['gene', 'note', 'codon_start', 'product', 'protein_id', 'translation'])
CDS Feature 13: dict_keys(['gene', 'codon_start', 'product', 'protein_id', 'translation'])
CDS Feature 15: dict_keys(['gene', 'codon_start', 'product', 'protein_id', 'translation'])
CDS Feature 17: dict_keys(['gene', 'codon_start', 'product', 'protein_id', 'translation'])
CDS Feature 19: dict_keys(['gene', 'note', 'codon_start', 'product', 'protein_id', 'translation'])
CDS Feature 21: dict_keys(['gene', 'codon_start', 'product', 'protein_id', 'translation'])


In [14]:
# Let's looks at the first CDS and see what each of these qualifiers are
nFeature = 5
for key, value in extractedRecord[0].features[nFeature].qualifiers.items():
    print(f"{key}: {value}")

gene: ['S']
note: ['structural protein']
codon_start: ['1']
product: ['surface glycoprotein']
protein_id: ['QHD43416.1']
translation: ['MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPL

In [15]:
# Let's also add the start site, stop site, lengths, and frame of the coding regions to our information:
nFeature = 3
for key, value in extractedRecord[0].features[nFeature].qualifiers.items():
    print(f"{key}: {value}")
print(f"Start: {extractedRecord[0].features[nFeature].location.start}")
print(f"Stop: {extractedRecord[0].features[nFeature].location.end}")
print(f"Nt Length: {len(extractedRecord[0].features[nFeature].location)}")
print(f"Frame: {extractedRecord[0].features[nFeature].location.strand}")

gene: ['orf1ab']
ribosomal_slippage: ['']
note: ['pp1ab; translated by -1 ribosomal frameshift']
codon_start: ['1']
product: ['orf1ab polyprotein']
protein_id: ['QHD43415.1']
translation: ['MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAFASEAARVVRSIFSRTLETAQNSVRVLQKAAITILDGISQYSLRLIDAMMFTSDLATNNLVVMAYITGGVVQLTSQWLTNIFGTVYEKLKPVLDWLEEKFKEGVEFLRDGWEIVKFISTCACEIVGGQIVTCAKEIKESVQTFFKLVNKFLALCADSIIIGGAKLKALNLGETFVTHSKGLYRKCVKSREETGLLMPLKAPKEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGTPVCINGLMLLEIKDTEKYCALAPNMMVTN

## **2.6) Getting more information online**

In [16]:
# Let's get more information by going directly to the sequence entry on NCBI.
sequence_link = 'https://www.ncbi.nlm.nih.gov/nuccore/' + corona_virus.annotations['accessions'][0]
webbrowser.open_new(sequence_link)

True

In [17]:
# We can get even more information about the sequence by looking 
# at the original paper that described this sequence.  For this, 
# we will first need the PUBMED ID number:
print(corona_virus.annotations['references'][0].pubmed_id)


32015508


In [18]:

# With that we can launch a browser and find the article:
pubmed_link = 'https://www.ncbi.nlm.nih.gov/pubmed/' + corona_virus.annotations['references'][0].pubmed_id
webbrowser.open_new(pubmed_link)

True

## **3) Simple analyses of the genomic sequence**

Now that we have the sequence data, let's do some simple analyses of the sequence:
1. Find the length of the sequence
2. Find the number of each nucleotide, dimer, trimer in the sequence
3. Find the number of each codon in the sequence
4. Find the number of open reading frames in the sequence
5. Find the largest open reading frame in the sequence
6. Translate the sequence into a protein sequence

## **3.1) Find the length, number of nucleotides, dimers, and trimers in the sequence**

In [ ]:
# Find the length of the sequence
covidLength = len(corona_virus.seq)  # Find the length of the sequence.
print(f'The length of the COVID19 sequence is {covidLength} nucleotides.')

In [ ]:
# Let's make a pie chart of the number of each nucleotide:

seq = str(corona_virus.seq)
neucleotides = 'ATGC'
counts = []
for i in range(4):
    counts.append(seq.count(neucleotides[i]))

plt.figure(figsize=(2.5, 2.5), dpi=200)
plt.pie(counts, labels=neucleotides, autopct='%1.1f%%')
plt.title('Nucleotide Counts')
plt.show()

In [ ]:
# Let's do the same thing, but fnow make a bar chart for the dimers
neucleotides = 'ATGC'
labels = []
dimerDict = {}
for i in range(4):
    for j in range(4):
        labels.append(neucleotides[i]+neucleotides[j])
        dimerDict[neucleotides[i]+neucleotides[j]] = 0
for i in range(len(corona_virus.seq)-1):
    dimerDict[corona_virus.seq[i:i+2]] += 1

# Check that the sum is correct
print(f'the sum of the dimers is = {sum([dimerDict[i] for i in dimerDict])}')
print(f'the length - 1 of the sequence is = {len(corona_virus.seq)-1}')  

plt.figure(figsize=(5, 2.5), dpi=200)
plt.bar(labels, [dimerDict[i] for i in dimerDict])
plt.xticks(rotation=90)
plt.title('Dimer Counts')
plt.show()


In [ ]:
# Now let's do the same for codons (i.e., triplets of nucleotides)
neucleotides = 'ATGC'
labels = []
codonDict = {}
for i in range(4):
    for j in range(4):
        for k in range(4):
            labels.append(neucleotides[i]+neucleotides[j]+neucleotides[k])
            codonDict[neucleotides[i]+neucleotides[j]+neucleotides[k]] = 0
for i in range(len(corona_virus.seq)-2):
    codonDict[corona_virus.seq[i:i+3]] += 1

# Check that the sum is correct
print(f'the sum of the codons is = {sum([codonDict[i] for i in codonDict])}')
print(f'the length - 2 of the sequence is = {len(corona_virus.seq)-2}')

plt.figure(figsize=(10, 2.5), dpi=200)
plt.bar(labels, [codonDict[i] for i in codonDict])
plt.xticks(rotation=90)
plt.title('Codon Counts')
plt.show()


### **3.2) Visualizing open reading frames in the sequence**

An **open reading frame** (ORF) is a sequence of DNA that can be translated into a protein. All proteins are made up of amino acids, and each amino acid is coded for by a sequence of three nucleotides in the DNA. An ORF is a sequence of DNA that starts with a **start codon** (ATG) and ends with a **stop codon** (TAA, TAG, or TGA).  

Because codons are three nucleotides long, there are three possible reading frames in a DNA sequence: the first reading frame starts at the first nucleotide, the second reading frame starts at the second nucleotide, and the third reading frame starts at the third nucleotide.  It is helpful to separate ORFs into these three reading frames to help identify potential protein-coding sequences.

In the next cell, we examine a sequence to find what its sequence would translate to in amino acids, and then we plot all of the open reading frames (ORFs) in each frame.

In [ ]:
# Print the AA sequence that would be coded in a given frame
frame = 0
AA_frame = corona_virus[frame:].translate(to_stop=False)
print(AA_frame.seq[1:1000])

Note that the sequence is read in the 5' to 3' direction, so the first reading frame starts at the first nucleotide, the second reading frame starts at the second nucleotide, and the third reading frame starts at the third nucleotide.

Let's write a function that will search the amino acid sequence for the start and stop codons and return the open reading frames.

In [23]:
# Now, let's make a plot of the open reading frames to show where they are on the sequence.
# Function to find ORFs in all frames of a sequence
def find_all_orfs(sequence):
    '''Find all open reading frames in all three reading frames of a sequence

    Inputs: 
        sequence (Bio.Seq) - The sequence to search for ORFs
    Output: 
        orf_list (list) - A list of lists of tuples. Each list of tuples contains the start 
        and stop positions of ORFs in a given frame.
    '''

    orf_list = []           # List of all the ORF we find
    for frame in range(3):  # Check each of the three reading frames
        orfs = []           # List of ORFs for the current frame   
        lastStop = frame    # Position of last stop codon.
        for orf in sequence[frame:].translate(to_stop=False).split('*'):     # Translate the curent frame sequence into AA and break up by stop codons
            if len(orf) > 30:                                                # Consider ORFs longer than 50 amino acids
                start = lastStop + orf.find('M') * 3 if 'M' in orf else None # Update start codon position to next M, 
                stop = lastStop + (len(orf))*3 if 'M' in orf else None       # Update stop codon to next '*'
                if start is not None and stop is not None:                   # In the coding region is >30AA
                    orfs.append((start, stop))                                      # Append the start and stop positions for that ORF
            lastStop += len(orf)*3 + 3           # Update the position of the last stop codon
        orf_list.append(orfs)                    # Append the list of ORFs for the current frame
    return orf_list

Now, let's use our ORF finding function to find the open reading frames and count the number of ORFs in each reading frame.

In [ ]:
# Find all ORFs in each frame of the sequence
orf_positions = find_all_orfs(corona_virus.seq)

for i in range(3):
    print(f'Number of ORFs in frame {i} = {len(orf_positions[i])}')

Let's plot the ORFs in each reading frame.

In [ ]:
# Visualize ORFs on the sequence for each frame
plt.figure(figsize=(15, 3))
colors = ['blue','black','purple']
for frame, orfs in enumerate(orf_positions):
    for start, stop in orfs:
        plt.plot([start, stop], [frame, frame], marker='o', color=colors[frame], markersize=5)
plt.title('Open Reading Frames in Each Frame')
plt.xlabel('Position')
plt.ylabel('Frame')
plt.yticks([0, 1, 2], ['Frame 0', 'Frame 1', 'Frame 2'])
plt.show()

We can see that there are many ORFs in each reading frame, but most of them are very short. This is because the start and stop codons are not evenly distributed throughout the sequence, so most of the ORFs are not full-length protein-coding sequences.

Let's write a function find the N longest ORF in a sequence.

In [26]:
def pickNLargestORF(orfs,N=5):
    '''Find the N largest ORFs in a list of ORFs

    Inputs: 
        orfs (list) - A list of lists of tuples. Each list of tuples contains the start 
        and stop positions of ORFs in a given frame.
        N (int) - The number of largest ORFs to return

    Output:
        Bounds (list) - A list of the start and stop positions of the N largest ORFs
    '''

    Lengths = []
    Bounds = []
    Frames = []
    for frame, orf in enumerate(orfs):
        for start, stop in orf:
            Lengths.append(stop-start)
            Bounds.append([start,stop])
            Frames.append(frame)
    enumerated_list = list(enumerate(Lengths))
    sortedLengths = sorted(enumerated_list, key=lambda x: abs(x[1]), reverse=True)
    
    # Extract lengths and bounds for those indices
    Bounds = [Bounds[index] for index, _ in sortedLengths[:N]]
    sortedFrames = [Frames[index] for index, _ in sortedLengths[:N]]
    return Bounds, sortedFrames

Now let's find and plot just the 10 longest ORFs.

In [ ]:
# Call our function to get the 10 longest ORFs.
longestORFbounds, frames = pickNLargestORF(orf_positions,N=10)

# Visualize the 10 longest ORFs on the sequence
plt.figure(figsize=(15, 3))
colors = ['blue','black','purple']
for i, (start, stop) in enumerate(longestORFbounds):
    plt.plot([start, stop], [frames[i], frames[i]], marker='o', color=colors[frames[i]], markersize=5)
plt.title('Longest Open Reading Frames')
plt.xlabel('Position')
plt.ylabel('Frame')
plt.yticks([0], ['Frame 0'])
plt.show()


## **3.2) Translating the sequence into a protein sequence**

Now that we have found the ORFs in the sequence, we can translate the sequence into a protein sequence. This is done by translating each codon in the DNA sequence into the corresponding amino acid.  Once we have the protein sequence, we can analyze it to find the number of each amino acid, the molecular weight of the protein, and the isoelectric point of the protein.

In [28]:
# Now let's look at the protein sequence and its properties
# Find the start and stop for longest ORF.
start, stop = longestORFbounds[0]

# Extract the nucleotide sequence for the longest ORF
longestORF = corona_virus.seq[start:stop]

# Translate the nucleotide sequence to protein
longestORF_AA = longestORF.translate(to_stop=True)

In [ ]:
# Count the number of amino acids in the protein and make a bar chart of the amino acid counts.
AAset = set(longestORF_AA)
AAcounts = {}
for AA in AAset:
    AAcounts[AA] = longestORF_AA.count(AA)

plt.figure(figsize=(5, 2.5), dpi=200)
plt.bar(AAcounts.keys(), AAcounts.values())
plt.title('Amino Acid Counts')
plt.show()

Next we are going to examine some properties of the protein sequence.  To do this, we are going to use the function ```ProteinAnalysis``` from the module ```Bio.SeqUtils.ProtParam```.  First, let's see what properties we can calculate for the protein sequence.

In [ ]:
# List available functions in ProtParam
protFeatures= ProtParam.ProteinAnalysis(str(longestORF_AA))
[x for x in dir(protFeatures) if not x.startswith('_')]

In [ ]:
# Compute the molecular weight of the protein
protFeatures= ProtParam.ProteinAnalysis(str(longestORF_AA))
protFeatures.molecular_weight()
print(f'The molecular weight of the protein is {protFeatures.molecular_weight()} Da.')

# Compute the isoelectric point of the protein
protFeatures.isoelectric_point()
print(f'The isoelectric point of the protein is {protFeatures.isoelectric_point()}.')

# Compute the instability index of the protein
protFeatures.instability_index()
print(f'The instability index of the protein is {protFeatures.instability_index()}.')

## 3.3) Comparing ORFs to Annotated Genes and Coding Sequences

Now let's see how our ORF's correspond to the annotated genes in the genbank file that we analyzed in Section 2.5.  To do this, we are just going to show the ORFs and the annotated genes in the same plot.

In [ ]:
# Let's update our figure of the ORFs to shopw both the annotated and found ORFs
plt.figure(figsize=(15, 3))
colors = ['blue','black','purple']
for i, (start, stop) in enumerate(longestORFbounds):
    plt.plot([start, stop], [frames[i], frames[i]], marker='o', color=colors[frames[i]], markersize=5)

for feature in extractedRecord[0].features:
    if feature.type == 'CDS' or feature.type == 'gene':
        start = int(feature.location.start)
        stop = int(feature.location.end)
        # check which frame the feature is in
        frame = start % 3
        plt.plot([start, stop], [frame+0.1, frame+0.1], marker='o', color='red', markersize=5, linestyle='dashed')


In [ ]:
# Let's see if we can get a little more information about our found ORFS using the annotated 
# features.

ORF_annotations = []
for i, (start, stop) in enumerate(longestORFbounds):
    ORF_annotation = None
    for feature in extractedRecord[0].features:
        if feature.type == 'CDS':
            featureStart = int(feature.location.start)            
            if start ==  featureStart:
                ORF_annotation = feature.qualifiers
                break
    ORF_annotations.append(ORF_annotation)

for i, annotation in enumerate(ORF_annotations):
    print(f'Annotation for ORF {i}: {annotation}')

In [ ]:
# Finally, let's update our figure with the annotations
plt.figure(figsize=(15, 3))
colors = ['blue','black','purple']
for i, (start, stop) in enumerate(longestORFbounds):
    plt.plot([start, stop], [frames[i], frames[i]], marker='o', color=colors[frames[i]], markersize=5)
    if ORF_annotations[i] is not None:
        plt.text(start, frames[i]-0.2, f"{ORF_annotations[i]['gene']}", fontsize=8)
        try:
            plt.text(start, frames[i]-0.35, f"{ORF_annotations[i]['product']}", fontsize=8)
        except:
            pass
        try:
            plt.text(start, frames[i]-0.5, f"{ORF_annotations[i]['note']}", fontsize=8)
        except:
            pass

for feature in extractedRecord[0].features:
    if feature.type == 'CDS' or feature.type == 'gene':
        start = int(feature.location.start)
        stop = int(feature.location.end)
        # check which frame the feature is in
        frame = start % 3
        plt.plot([start, stop], [frame+0.1, frame+0.1], marker='o', color='red', markersize=5, linestyle='dashed')

# set y limit
plt.ylim(-.5, 2.2)